In [ ]:
#default_exp synchro.processing

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import numpy as np
import datetime
import glob
import os

In [ ]:
#export
def get_thresholds(data):
    max_val = max(data[len(data)//2:len(data)//2 + 10000000]) #Looking for a max in a portion of the data, from the middle
    high_thresh = max_val*3/4 # High threshold set at 3/4th of the max
    low_thresh  = max_val*1/4
    return low_thresh, high_thresh

In [ ]:
#export
def get_first_high(data, threshold):
    if np.any(data>threshold):
        return np.argmax(data>threshold)
    else:
        return -1
    
def reverse_detection(data, frame_timepoints, low_threshold, increment):
    new_timepoints = []
    new_signals = []

    safe_increment = int(increment * 105/100)

    i = frame_timepoints[0]-safe_increment
    while i>0:
        data_slice = data[i:i+increment//2]
        if np.any(data_slice > low_threshold):
            i = i+np.argmax(data_slice > low_threshold)
        else:
            break #No low threshold crossing found -> no more frames to detect
        new_timepoints.append(i)
        i-= safe_increment #We move backward of almost a frame

    return new_timepoints[::-1]
    
def extend_timepoints(frame_timepoints, n=10):
    frame_timepoints = np.array(frame_timepoints)
    typical_distance = int(np.mean(np.diff(frame_timepoints)))
    extended_tp = [frame_timepoints[0]-(i+1)*typical_distance for i in range(n) if (frame_timepoints[0]-(i+1)*typical_distance)>0]
    return extended_tp[::-1]
    
def detect_frames(data, low_threshold, high_threshold, increment):
    frame_timepoints, frame_signals = [], []
    safe_increment = int(increment*95/100)

    first_high = get_first_high(data, high_threshold)
    if first_high == -1:
        print("No high frame detected. Detection can't work.")
        return

    frame_timepoints.append(first_high)
    frame_signals.append(1)

    new_timepoints   = reverse_detection(data, frame_timepoints, low_threshold, increment)
    new_extrapolated = extend_timepoints(new_timepoints)
    frame_timepoints = new_extrapolated + new_timepoints + frame_timepoints
    frame_signals    = [0]*(len(new_timepoints)+len(new_extrapolated)) + frame_signals

    i = first_high + safe_increment
    while i < len(data):
        data_slice = data[i:i+increment//2]
        if np.any(data_slice>low_threshold):
            i = i+np.argmax(data_slice>low_threshold)
        else:
            break #This frame sequence is over. Pass the next sequence through this function if there are frames left
        frame_timepoints.append(i)
        frame_signals.append(int(np.any(data_slice > high_threshold)))
        i += safe_increment

    frame_timepoints = np.array(frame_timepoints)
    frame_signals    = np.array(frame_signals)
    frame_timepoints = frame_timepoints - 3 # A slight shift of the timepoints 
                                            # to include the begginning of the peaks.

#     self.frame_start_time = self.record_start_time + datetime.timedelta(0,int(self.frame_timepoints[0]/self.sampling_rate))
#     self.frame_end_time = self.record_start_time + datetime.timedelta(0,int(self.frame_timepoints[-1]/self.sampling_rate))

    return frame_timepoints, frame_signals

In [ ]:
#export
def cluster_frame_signals(data, frame_timepoints, n_cluster=5):
    frame_aucs = np.fromiter(map(np.trapz, np.split(data, frame_timepoints)), float)
    if frame_timepoints[0] != 0: #We need to remove the first part if it wasn't a full frame
        frame_aucs = frame_aucs[1:]
    frame_auc_sorted = np.sort(frame_aucs)
    deriv = np.array(frame_auc_sorted[1:]-frame_auc_sorted[:-1])
    deriv[:5]  = 0 #removing tails values that can show weird stuff
    deriv[-5:] = 0
    n          = n_cluster - 1
    idx_gaps = np.zeros(n, dtype="int")
    tmp_deriv = deriv.copy()
    for i in range(n):
        idx_gaps[i] = np.argmax(tmp_deriv)
        tmp_deriv[idx_gaps[i]] = 0
    idx_gaps = np.sort(idx_gaps)
    thresholds = np.zeros(n, dtype="float")
    for i, idx in enumerate(idx_gaps):
        thresholds[i] = (frame_auc_sorted[idx+1] + frame_auc_sorted[idx])/2

    return np.array([np.sum(auc>thresholds) for auc in frame_aucs], dtype=int)

In [ ]:
#export
def parse_time(time_str, pattern="%y%m%d_%H%M%S"):
    return datetime.datetime.strptime(time_str, pattern)

def get_position_estimate(stim_time, record_time, sampling_rate):
    if stim_time < record_time:
        return -1
    else:
        return (stim_time - record_time).seconds * sampling_rate

In [ ]:
#export
def match_starting_position(frame_timepoints, frame_signals, stim_signals, estimate_start):
    stim_matching_len = min(600, np.where(np.diff(stim_signals)!=0)[0][50]) #Way of getting the 50th change in the signals
    #But not higher than 600 (correspond to 10s, and is necessary for moving gratings)
#     stim_matching_len = 50
    idx_estimate = np.argmax(frame_timepoints>estimate_start)
    search_slice = slice(max(0, idx_estimate-1000), min(idx_estimate+1000, len(frame_signals)))
#     diff_signals = np.diff(frame_signals[search_slice])
#     diff_stim    = np.diff(stim_signals[:stim_matching_len])
#     return search_slice.start + np.argmax(np.correlate(diff_signals, diff_stim))
    return search_slice.start + np.argmax(np.correlate(frame_signals[search_slice], 
                                                       stim_signals[:stim_matching_len]))

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted 02_processing.ipynb.
Converted 03_modelling.ipynb.
Converted 04_plotting.ipynb.
Converted 05_database.ipynb.
Converted 10_synchro.io.ipynb.
Converted 11_synchro.extracting.ipynb.
Converted 12_synchro.processing.ipynb.
Converted 99_testdata.ipynb.
Converted index.ipynb.
